In [1]:
import pandas as pd
df = pd.read_parquet("../output/communities.parquet")

In [2]:
df.dtypes

id                   object
human_readable_id     int64
community             int64
level                 int64
parent                int64
children             object
title                object
entity_ids           object
relationship_ids     object
text_unit_ids        object
period               object
size                  int64
dtype: object

In [3]:
df.shape

(366, 12)

In [4]:
df.head()

,id,human_readable_id,community,level,parent,children,title,entity_ids,relationship_ids,text_unit_ids,period,size
0,1e0797fa-58fb-4b3b-b7cd-885cee73b6e2,0,0,0,-1,"[35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46]",Community 0,"[8717d0c8-06ed-4949-a931-445aa403d808, 808c7d3...","[01d52357-a9bb-4335-90b5-7430c0adea7e, 05f7e79...",[01128fddedc15ddb88152339acf19e969444d4cf2afa8...,2026-02-08,116
1,a573dfa9-f15c-49b3-9894-9f003404d921,1,1,0,-1,"[47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 5...",Community 1,"[94d0afe6-ce6d-4d99-8eb3-8b90a756ed37, 0dc549d...","[00d2b4ef-f46f-49a4-b6d2-cd5d047d70c1, 03d5f29...",[01128fddedc15ddb88152339acf19e969444d4cf2afa8...,2026-02-08,169
2,607970c6-c495-4f31-8a04-52325edcba19,2,2,0,-1,"[67, 68, 69, 70, 71, 72, 73, 74, 75, 76]",Community 2,"[35c090a2-e65e-4994-8e7e-367356014dd6, 6ac49ff...","[014f0b74-3f7c-4890-8cb7-c73caa10916a, 02bc9d8...",[13c0038e63fa1b31b599e9608433910741790eae9e098...,2026-02-08,66
3,c5ad0ede-a8de-4efa-a67d-9dc464567a83,3,3,0,-1,"[77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 8...",Community 3,"[961bd800-15c2-4042-abbb-a0bb1fd1b964, 7413b71...","[018c20bb-b3e2-48c8-bb82-a63fb9f3382d, 03c6a61...",[03de41385cca02cab7e3fef5643427a757b8ccd71d827...,2026-02-08,190
4,cd382a2b-8214-4c9a-83c2-6c2c457d2b34,4,4,0,-1,"[96, 97, 98, 99, 100, 101, 102, 103, 104, 105,...",Community 4,"[c2632fcc-934c-4cc7-93f9-95b933ef06ac, e7e70d9...","[0024ec3b-5a6b-4dd8-ab20-6a893c96f633, 004ec58...",[01128fddedc15ddb88152339acf19e969444d4cf2afa8...,2026-02-08,236


In [5]:
with pd.option_context(
    "display.max_colwidth", None,
    "display.max_columns", None,
    "display.width", None,
):
    row = df.iloc[0]
    print(f"id={row['id']}")
    print("children:")
    print(f"\t{row['children']}")
    print("text_unit_ids:")
    print(f"\t{row['text_unit_ids']}")
    

id=1e0797fa-58fb-4b3b-b7cd-885cee73b6e2
children:
	[35 36 37 38 39 40 41 42 43 44 45 46]
text_unit_ids:
	['01128fddedc15ddb88152339acf19e969444d4cf2afa8d9a06f489811bbff330299790a88c3e40f4c1c04d727658de7d82108b74468bede46567ca0d422491ea'
 '0c13ebc8ab4bd202e1e4b54b0173404bb98ac80c7208940eb2224e075d48b6ec4612db75e112ea38dd27188fda55cf7e80e1c9dce702358888ab46924ea3bf53'
 '11db0112a3ad47cb15df74759597c2f0a0c42c9c082a80df5791d0d055d702739b27fcc6a971b2a2f01af6e734e0943d57ba1a23dfc6e78dc1bbf43a05647795'
 '13c0038e63fa1b31b599e9608433910741790eae9e098981a0676343b407207407ed7a238d623561c04924ea0ce8af89453097e5388bc6998c932faa209715ec'
 '143159adfddd9e5576dde94769c60d5d96073fdcbc6a0cf8c1bdc8464b9911ac36a8717632e8a074c61d7d5510b5e977b42476b6bfc69c823389eec591e4b9db'
 '1545db09ebf4593acd7e5a134c475656bcd2e9d28c1d688914153dda99d5dd1883c7443521c2f9b78c8a6300996dcdc1b61d94dae02322b0c4c52f1db07189ea'
 '17760b7f21c9ec45b0124dffab78ae6d99f1dfa3fb8bef61fd52482d07e6b7150ba389824463cfded79c173b1cc370348f72b

In [6]:
len(df[df["parent"] == -1])

35

In [7]:
# ルート（parent == -1）から human_readable_id で階層を辿る
levels = []
current = df[df["parent"] == -1]
level = 0
levels.append((level, len(current)))

while len(current) > 0:
    next_ids = current["human_readable_id"].tolist()
    current = df[df["parent"].isin(next_ids)]
    level += 1
    levels.append((level, len(current)))

for lvl, cnt in levels:
    if lvl == 0:
        print(f"level {lvl} (root): {cnt}")
    else:
        print(f"level {lvl}: {cnt}")

level 0 (root): 35
level 1: 203
level 2: 126
level 3: 2
level 4: 0
